In [35]:
import numpy as np

In [36]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   웨이브 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 웨이브 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)

batch_size 개수만큼의 시계열 데이터가 생성

각 시계열 데이터는 n_steps 만큼의 데이터가 있고,

앞선 n_steps - 1 개의 데이터를 통해 n_steps 번째의 데이터를 예측한다.

In [37]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [38]:
X_train.shape

(7000, 50, 1)

In [39]:
X_train[0].shape

(50, 1)

In [40]:
y_train.shape

(7000, 1)

In [41]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):
    return x * (1 - x)

In [42]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self):
    return self.predict - self.target

In [233]:
class RNN_vector_to_sequence:
    #입력값
    input_data = []
    
    #가중치
    weight = []
    
    #편향값
    bias = []

    # 입력 가중치
    input_w = []

    # 이전 출력에 대한 가중치
    before_w = []

    # 출력 가중치
    result_w = []

    # 편향 값 bais
    bias = []

    # 노드 입력
    node_input = []

    # 노드 출력
    node_output = []

    # 타겟, 목푯값
    target = []

    # 활성화 함수
    activation = activation_function()

    # 비용 함수
    cost = cost_function()

    # 예측 결과
    predict = []

    # 각 층의 델타 값의 저장
    delta = []

     # 각 가중치 업데이트 크기
    input_weight_update = []

    before_weight_update = []

    result_weight_update = []

    def delta_to_sequence_cal_rnn(self, input, target):

      # 타겟값의 저장
      self.target = target


      # 입력 데이터 크기에 맞는 가중치 값이 생성되어야 한다.
      # 입력 (3,1) 데이터에 대한 연산 결과 (1,1)를 예측한다고 가정,

      # 입력값에 사용되는 가중치
      # (3,1) 크기의 입력 데이터에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위해, (1,3) 크기의 가중치 행렬이 필요하다
      input_w = np.random.rand(target.shape[0], input.shape[0])

      # 이전 출력에 대한 가중치
      # (1,1) 노드 출력에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위한, (1,1) 크기의 가중치 행렬
      before_w = np.random.rand(target.shape[0], target.shape[0])

      # 예측에 사용되는 가중치
      # 노드 출력 (3,1) 에 대해 예측값 (1,1) 크기의 데이터 행렬을 얻기 위해, (1,3) 크기의 가중치 행렬이 필요하다.
      result_w = np.random.rand(target.shape[0], input.shape[0])

      # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
      before = np.zeros((target.shape[0], target.shape[0]))

      # (3, n) 크기의 입력 데이터에서 n 번의 반복을 수행하게 됨
      for i in range(input.shape[0]):
        # 노드 입력 계산
        node_input = (before_w @ before) + (input_w @ input)
        self.node_input.append(node_input)

        # 노드 출력 계산
        node_output = self.activation.sigmoid(node_input)
        self.node_output.append(node_output)

        # 순환 노드의 다음 노드 입력값의 계산
        before = self.before_w @ node_output

      self.predict = self.result_w @ node_output

      return self.predict

    def delta_to_sequence_cal_rnn_test(self, input, target):

      # 타겟값의 저장
      self.target = target

      # 입력 데이터의 저장
      self.input = input

      # 입력 데이터 크기에 맞는 가중치 값이 생성되어야 한다.
      # 입력 (3,1) 데이터에 대한 연산 결과 (1,1)를 예측한다고 가정,

      # 입력값에 사용되는 가중치
      # (3,1) 크기의 입력 데이터에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위해, (1,3) 크기의 가중치 행렬이 필요하다
      self.input_w = np.random.rand(target.shape[0], input.shape[1])
      
      # 이전 출력에 대한 가중치
      # (1,1) 노드 출력에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위한, (1,1) 크기의 가중치 행렬
      self.before_w = np.random.rand(target.shape[0], target.shape[0])

      # 예측에 사용되는 가중치
      # 노드 출력 (3,1) 에 대해 예측값 (1,1) 크기의 데이터 행렬을 얻기 위해, (1,3) 크기의 가중치 행렬이 필요하다.
      self.result_w = np.random.rand(target.shape[0], input.shape[1])

      # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
      before = np.zeros((target.shape[0], target.shape[0]))
      
      # (3, n) 크기의 입력 데이터에서 n 번의 반복을 수행하게 됨
      for i in range(input.shape[0]):
        # 노드 입력 데이터의 numpy 행렬 생성
        input_data = input[i].reshape(input.shape[1],-1)

        # RNN 노드 입력 값 계산, 이전 노드 출력 @ 가중치 + 입력 데이터 @ 가중치
        node_input = (self.before_w @ before) + (self.input_w @ input_data)
        self.node_input.append(node_input)

        # 노드 출력 계산, 활성화 함수 연산을 수행한다.
        node_output = self.activation.sigmoid(node_input)
        self.node_output.append(node_output)

        # 순환 노드의 다음 노드 입력값의 계산
        before = self.before_w @ node_output

      self.predict = self.activation.sigmoid(self.result_w @ node_output)

      return self.predict

    def cost_cal(self):
      return self.cost.errer_squared_sum(self.predict, self.target)

    
    def cal_delta(self):
      #출력 노드의 변화량에 대한 오차 함수의 변화량 계산
      delta = (self.cost.diff_error_squared_sum() * self.activation.sigmoid_diff(self.predict))
      self.delta.append(delta)
      print(delta)
      
      # 해당 delta 값을 통해 result_w 의 가중치 업데이트 크기를 계산할 수 있다.
      # delta 값과 result_w 의 변화량(이전 노드의 출력) 의 계산을 통해 result_w 의 변화량을 계산한다.
      self.result_weight_update = (delta @ self.node_output[self.input.shape[0] - 1].T).T
      print(self.result_weight_update, self.input.shape[0] - 1)

      # 노드 출력에 대한 변화량 계산, 벡터 투 시퀀스 RNN 에서는 result_w 가중치가 한 번만 사용된다.
      # node_output_diff 는 출력층 이전 노드 출력의 변화량에 대한 비용 함수의 변화량
      node_output_diff = (delta @ self.result_w)
      print(node_output_diff) 
      
      # 활성화 함수 변화량 적용
      node_output_diff_activation = node_output_diff * self.activation.sigmoid_diff(self.node_input[self.input.shape[0] - 1])
      print(node_output_diff_activation)

      # 노드 입력에 대한 비용 함수의 변화량
      node_input_diff = self.node_output[self.input.shape[1] - 2] @ node_output_diff_activation 

      # n 개의 delta 값을 계산하기 위해 각 반복은 입력 데이터의 개수(3, n) 의 n 번 만큼 반복되어야 한다.
      for i in range(len(self.input.shape[1]) - 2, -1, -1):
        
        activation_diff = self.activation.sigmoid_diff(delta)
        # 


In [234]:
rnn = RNN_vector_to_sequence()

In [235]:
input = X_train[0]

In [236]:
target = y_train[0].reshape(1,-1)

In [237]:
input.shape

(50, 1)

In [238]:
target.shape

(1, 1)

In [239]:
rnn.delta_to_sequence_cal_rnn_test(input, target)

array([[0.57564077]])

In [240]:
rnn.cost_cal()

0.4329474602095212

In [241]:
rnn.cal_delta()

[[0.22730961]]
[[0.10906401]] 49
[[0.14444964]]
[[-0.01261943]]


TypeError: ignored

In [ ]:
rnn.node_output